<a href="https://colab.research.google.com/github/abhishekhbhootwala/MixlabPiash/blob/master/AIO_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://i.ibb.co/C1kJ50G/RDP.png" height="45" align="left" alt=""/><font size=6>Colab RDP</font>

In [ ]:
#@title ← Start VPS
import apt, apt.debfile
import pathlib, stat, shutil, urllib.request, subprocess, getpass, time, tempfile
import secrets, json, re
import IPython.utils.io
import ipywidgets
import os
from IPython.display import clear_output

class _NoteProgress(apt.progress.base.InstallProgress, apt.progress.base.AcquireProgress, apt.progress.base.OpProgress):
  def __init__(self):
    apt.progress.base.InstallProgress.__init__(self)
    self._label = ipywidgets.Label()
    display(self._label)
    self._float_progress = ipywidgets.FloatProgress(min = 0.0, max = 1.0, layout = {'border':'1px solid #118800'})
    display(self._float_progress)

  def close(self):
    self._float_progress.close()
    self._label.close()

  def fetch(self, item):
    self._label.value = "fetch: " + item.shortdesc

  def pulse(self, owner):
    self._float_progress.value = self.current_items / self.total_items
    return True

  def status_change(self, pkg, percent, status):
    self._label.value = "%s: %s" % (pkg, status)
    self._float_progress.value = percent / 100.0

  def update(self, percent=None):
    self._float_progress.value = self.percent / 100.0
    self._label.value = self.op + ": " + self.subop

  def done(self, item=None):
    pass

class _MyApt:
  def __init__(self):
    self._progress = _NoteProgress()
    self._cache = apt.Cache(self._progress)

  def close(self):
    self._cache.close()
    self._cache = None
    self._progress.close()
    self._progress = None

  def update_upgrade(self):
    self._cache.update()
    self._cache.open(None)
    self._cache.upgrade()

  def commit(self):
    self._cache.commit(self._progress, self._progress)
    self._cache.clear()

  def installPkg(self, *args):
    for name in args:
      pkg = self._cache[name]
      if pkg.is_installed:
        print(f"{name} is already installed")
      else:
        pkg.mark_install()

  def installDebPackage(self, name):
    apt.debfile.DebPackage(name, self._cache).install()

  def deleteInstalledPkg(self, *args):
    for pkg in self._cache:
      if pkg.is_installed:
        for name in args:
          if pkg.name.startswith(name):
            #print(f"Delete {pkg.name}")
            pkg.mark_delete()

def _download(url, path):
  try:
    with urllib.request.urlopen(url) as response:
      with open(path, 'wb') as outfile:
        shutil.copyfileobj(response, outfile)
  except:
    print("Failed to download ", url)
    raise

def _get_gpu_name():
  r = subprocess.run(["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"], stdout = subprocess.PIPE, universal_newlines = True)
  if r.returncode != 0:
    return None
  return r.stdout.strip()

def _check_gpu_available():
  gpu_name = _get_gpu_name()
  if gpu_name == None:
    print("This is not a runtime with GPU")
  else:
    return True

  return True

def _setupSSHDImpl(ngrok_token, ngrok_region, is_VNC):

  msg = ""

  if not os.path.exists('ngrok.zip'):
    _download("https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip", "ngrok.zip")
    shutil.unpack_archive("ngrok.zip")
    pathlib.Path("ngrok").chmod(stat.S_IXUSR)

  root_password = "12345678"
  user_password = "12345678"
  user_name = "colab"
  msg += "✂️"*24 + "\n"
  msg += f"root password: {root_password}\n"
  msg += f"{user_name} password: {user_password}\n"
  msg += "✂️"*24 + "\n"
  subprocess.run(["useradd", "-s", "/bin/bash", "-m", user_name])
  subprocess.run(["adduser", user_name, "sudo"], check = True)
  subprocess.run(["chpasswd"], input = f"root:{root_password}", universal_newlines = True)
  subprocess.run(["chpasswd"], input = f"{user_name}:{user_password}", universal_newlines = True)

  if not pathlib.Path('/root/.ngrok2/ngrok.yml').exists():
    subprocess.run(["./ngrok", "authtoken", ngrok_token])
  
  ngrok_proc = subprocess.Popen(["./ngrok", "tcp", "-region", ngrok_region, "5901"])
  time.sleep(2)

  with urllib.request.urlopen("http://localhost:4040/api/tunnels") as response:
    url = json.load(response)['tunnels'][0]['public_url']
    m = re.match("tcp://(.+):(\d+)", url)

  hostname = m.group(1)
  port = m.group(2)

  if is_VNC:
    msg += "Ready to Connect TurboVNC viewer:\n"
    msg += f"{hostname}:{port}\n"
  return msg

def _setupSSHDMain(ngrok_region, check_gpu_available, is_VNC):
  if check_gpu_available and not _check_gpu_available():
    return (False, "")

  gpu_name = _get_gpu_name()
  if gpu_name == None:
    print("------------------------------")

  print("Copy&paste your tunnel authtoken from https://dashboard.ngrok.com/auth")
  print("(You need to sign up for ngrok and login,)")
  #Set your ngrok Authtoken.
  ngrok_token = getpass.getpass()
  clear_output()

  if not ngrok_region:
    print("Select your ngrok region :")
    print("us - United States (Ohio)")
    print("eu - Europe (Frankfurt)")
    print("ap - Asia/Pacific (Singapore)")
    print("au - Australia (Sydney)")
    print("sa - South America (Sao Paulo)")
    print("jp - Japan (Tokyo)")
    print("in - India (Mumbai)")
    ngrok_region = region = input()
    clear_output()
    apps()

  return (True, _setupSSHDImpl(ngrok_token, ngrok_region, is_VNC))

def _setup_nvidia_gl():
  ret = subprocess.run(
                  ["nvidia-smi", "--query-gpu=driver_version", "--format=csv,noheader"],
                  stdout = subprocess.PIPE,
                  check = True,
                  universal_newlines = True)
  nvidia_version = ret.stdout.strip()
  nvidia_url = "https://us.download.nvidia.com/tesla/{0}/NVIDIA-Linux-x86_64-{0}.run".format(nvidia_version)
  _download(nvidia_url, "nvidia.run")
  pathlib.Path("nvidia.run").chmod(stat.S_IXUSR)
  subprocess.run(["./nvidia.run", "--no-kernel-module", "--ui=none"], input = "1\n", check = True, universal_newlines = True)

  #https://virtualgl.org/Documentation/HeadlessNV
  subprocess.run(["nvidia-xconfig",
                  "-a",
                  "--allow-empty-initial-configuration",
                  "--virtual=1920x1200",
                  "--busid", "PCI:0:4:0"],
                 check = True
                )

  with open("/etc/X11/xorg.conf", "r") as f:
    conf = f.read()
    conf = re.sub('(Section "Device".*?)(EndSection)',
                  '\\1    MatchSeat      "seat-1"\n\\2',
                  conf,
                  1,
                  re.DOTALL)

  with open("/etc/X11/xorg.conf", "w") as f:
    f.write(conf)

  subprocess.run(["/opt/VirtualGL/bin/vglserver_config", "-config", "+s", "+f"], check = True)
  subprocess.Popen(["Xorg", "-seat", "seat-1", "-allowMouseOpenFail", "-novtswitch", "-nolisten", "tcp"])

def _setupVNC():
  libjpeg_ver = "2.0.5"
  virtualGL_ver = "2.6.4"
  turboVNC_ver = "2.2.5"

  libjpeg_url = "https://github.com/demotomohiro/turbovnc/releases/download/2.2.5/libjpeg-turbo-official_{0}_amd64.deb".format(libjpeg_ver)
  virtualGL_url = "https://github.com/demotomohiro/turbovnc/releases/download/2.2.5/virtualgl_{0}_amd64.deb".format(virtualGL_ver)
  turboVNC_url = "https://github.com/demotomohiro/turbovnc/releases/download/2.2.5/turbovnc_{0}_amd64.deb".format(turboVNC_ver)

  _download(libjpeg_url, "libjpeg-turbo.deb")
  _download(virtualGL_url, "virtualgl.deb")
  _download(turboVNC_url, "turbovnc.deb")
  my_apt = _MyApt()
  my_apt.installDebPackage("libjpeg-turbo.deb")
  my_apt.installDebPackage("virtualgl.deb")
  my_apt.installDebPackage("turbovnc.deb")

  my_apt.commit()
  my_apt.close()

  vnc_sec_conf_p = pathlib.Path("/etc/turbovncserver-security.conf")
  vnc_sec_conf_p.write_text("""\
no-remote-connections
no-httpd
no-x11-tcp-connections
""")

  gpu_name = _get_gpu_name()
  if gpu_name != None:
    _setup_nvidia_gl()

  vncrun_py = tempfile.gettempdir() / pathlib.Path("vncrun.py")
  vncrun_py.write_text("""\
import subprocess, secrets, pathlib

vnc_passwd = "12345678"[:8]
vnc_viewonly_passwd = "87654321"[:8]
print("✂️"*24)
print("VNC password: {}".format(vnc_passwd))
print("VNC view only password: {}".format(vnc_viewonly_passwd))
print("✂️"*24)
vncpasswd_input = "{0}\\n{1}".format(vnc_passwd, vnc_viewonly_passwd)
vnc_user_dir = pathlib.Path.home().joinpath(".vnc")
vnc_user_dir.mkdir(exist_ok=True)
vnc_user_passwd = vnc_user_dir.joinpath("passwd")
with vnc_user_passwd.open('wb') as f:
  subprocess.run(
    ["/opt/TurboVNC/bin/vncpasswd", "-f"],
    stdout=f,
    input=vncpasswd_input,
    universal_newlines=True)
vnc_user_passwd.chmod(0o600)
subprocess.run(
  ["/opt/TurboVNC/bin/vncserver"],
  cwd = pathlib.Path.home()
)

#Disable screensaver because no one would want it.
(pathlib.Path.home() / ".xscreensaver").write_text("mode: off\\n")
""")
  r = subprocess.run(
                    ["su", "-c", "python3 " + str(vncrun_py), "colab"],
                    check = True,
                    stdout = subprocess.PIPE,
                    universal_newlines = True)
  return r.stdout

def setupVNC(ngrok_region = None, check_gpu_available = True):
  stat, msg = _setupSSHDMain(ngrok_region, check_gpu_available, True)
  if stat:
    msg += _setupVNC()
  clear_output()
  _check_gpu_available()
  print(msg)

def apps():
  !sudo add-apt-repository ppa:obsproject/obs-studio -y
  ! wget https://github.com/SaladTechnologies/salad-applications/releases/download/0.5.4/Salad_0.5.4_amd64.deb
  ! sudo dpkg -i Salad_0.5.4_amd64.deb
  ! sudo apt --fix-broken install
  !sudo apt-get install xfce4-terminal
  !sudo update-alternatives --config x-terminal-emulator

  clear_output()
  my_apt = _MyApt()
  my_apt.installPkg("obs-studio", "chromium-browser", "vlc", "firefox", "telegram-desktop", "qbittorrent", "uget", "xarchiver", "xfce4", "xfce4-goodies")
  my_apt.commit()
  clear_output()

setupVNC()

### <img src="https://i.ibb.co/W0Ry3Kt/Heart-Pulse.png" height="45" align="left" alt=""/><font size=6>Keep Alive</font>

In [ ]:
#@title ← Keep VPS Alive For 12 hours!
! sleep 9999999

In [ ]:
#@markdown <h3> ⬅ Prevent Google Colab from disconnecting | AUTO RECONNECT
import IPython
from google.colab import output
 
display(IPython.display.Javascript('''
function ClickConnect(){
  btn = document.querySelector("colab-connect-button")
  if (btn != null){
    console.log("Click colab-connect-button"); 
    btn.click() 
    }
    
  btn = document.getElementById('ok')
  if (btn != null){
    console.log("Click reconnect"); 
    btn.click() 
    }
  }
    
setInterval(ClickConnect,60000)
'''))

# <img src="https://i.ibb.co/pnknw4V/Dwonloader.png" height="45" align="left" alt=""/><font size=6>Downloader</font>

### <img src="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/aria2cli.png" height="45" alt="aira2"/> <font size=1px >___Direct link downloader___</font>

In [ ]:
URL = "" #@param {type:"string"}
#@markdown <center><h5>Default output path /content/downloads/</h5></center>
OUTPUT_PATH = "" #@param {type:"string"}

import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ocr.py"): 
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

def youtubedlInstall():
  if not path.isfile("/usr/local/bin/youtube-dl"):
    cmdC = "rm -rf /content/sample_data/ && " \
            " mkdir -p -m 666 /root/.YouTube-DL/ &&" \
            " apt-get install atomicparsley &&" \
            " curl -L https://yt-dl.org/downloads/latest/youtube-dl " \
            "-o /usr/local/bin/youtube-dl &&" \
            " chmod a+rx /usr/local/bin/youtube-dl"
    get_ipython().system_raw(cmdC)

def aria2Install():
  runSh('apt install -y aria2')

def istmd(URL): 
  link = urlparse(URL)
    
  #YandexDisk
  if link.netloc == "yadi.sk":
    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/' \
                    '?public_key={}&path=/{}&offset={}'
    dry = False
    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()


    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x 16 -s 16 -k 1M -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)


    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False

        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return

        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)

        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL

if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/downloads/"
  
if not URL == "":
  aria2Install()
  youtubedlInstall()
  try:
    mkdir("downloads")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    cmdC = f'youtube-dl -o "{OUTPUT_PATH}/%(title)s" {URL} ' \
            '--external-downloader aria2c ' \
            '--external-downloader-args "-x 16 -s 16 -k 1M"'
    runSh(cmdC, output=True)
else:
  print("Please input url")

In [ ]:
#@title #<img src="https://i.ibb.co/YWv2T2f/wget.png" height="45" align="left" alt=""/><font size=6>__Direct Downloader__</font> { vertical-output: true }
URL = "" #@param {type:"string"}
Download_Destination = "" #@param {type:"string"}
#@markdown <center><h5>Default output path /content/downloads/</h5></center>
if not Download_Destination:
  Download_Destination = "/content/downloads/"
url, downdir = URL, Download_Destination
!wget -c $url --no-check-certificate -P $downdir

In [1]:
#@title #<img src="https://i2.wp.com/clay-atlas.com/wp-content/uploads/2020/03/google-drive.png" height="45" align="left" alt=""/><font size=6>__Downloader Location__</font> { vertical-output: true }
import os
os.chdir("/content/drive/Shareddrives/Random")

In [ ]:
#@title #<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Circle-icons-download.svg/1024px-Circle-icons-download.svg.png" height="45" align="left" alt=""/><font size=6>__Drive Downloader__</font> { vertical-output: true }
#@markdown __<center><h5>Must Run Download Location Cell Before this Cell</h5></center>__
URL = "https://drive.google.com/u/0/uc?id=" #@param {type:"string"}
url = URL
!gdown $url

In [ ]:
#@title #<img src="https://icons-for-free.com/iconfiles/png/512/Wget-1324888774139318677.png" height="45" align="left" alt=""/><font size=6>__Direct Downloader__</font> { vertical-output: true }
#@markdown __<center><h5>Must Run Download Location Cell Before this Cell</h5></center>__
URL = "" #@param {type:"string"}
url = URL
!curl -C - -O -J -L --header 'Connection: keep-alive' $url

In [ ]:
#@title #__⬇️ Download From Direct Link__#
#@markdown __<center><h5>Must Run Download Location Cell Before this Cell</h5></center>__
URL = "" #@param {type:"string"}
!wget -c "$URL" --no-check-certificate

### <img src="https://i.loli.net/2019/08/12/IKCBpfQ4yzSqFdE.png" height="45" align="left" alt=""/><font size=6>Aria2</font>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] aria2</font>
Aria2_rpc = True
Ariang_WEBUI = True
USE_FREE_TOKEN = True
TOKEN = ""
REGION = "US"
Tunneling = "argo_tunnel_(cloudflare)" #@param ["argo_tunnel_(cloudflare)", "localhost.run", "ngrok"]
# OUTPUT_DIR = "" # @param {type:"string"}

if Tunneling == "argo_tunnel_(cloudflare)":
    PORT_FORWARD = "argotunnel"
elif Tunneling == "localhost.run":
    PORT_FORWARD = "localhost"
elif Tunneling == "ngrok":
    PORT_FORWARD = "ngrok"
# ================================================================ #

import os, pathlib, zipfile, re
import urllib.request, requests
from IPython.display import HTML, clear_output
from urllib.parse import urlparse

PORT = 8221
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
            "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    CWD,
    displayUrl,
    findProcess,
    findPackageR
)

loadingAn()

# Setting up aria2
runSh('apt install -y aria2')
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)

# Defining Github latest release tag
def latestTag(link):
    import re
    from urllib.request import urlopen
    htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
    return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]

# Downloading the latest version of ariaNg
if not os.path.exists("ariang/index.html"):
    # BASE_URL = r"https://github.com/mayswind/AriaNg"
    # LATEST_TAG = latestTag(BASE_URL)
    # urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
    #        f'AriaNg-{LATEST_TAG}-AllInOne.zip'
    urllib.request.urlretrieve(findPackageR('mayswind/AriaNg', 'AllInOne.zip'), 'ariang/new.zip')
    with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref: zip_ref.extractall('ariang')
    try:
        pathlib.Path('ariang/new.zip').unlink()
    except FileNotFoundError:
        pass

# Starting up aria2 RPC and the WebUI (ariaNg)
try:
    if not OUTPUT_DIR:
        OUTPUT_DIR = f"downloads/"
    elif not os.path.exists(OUTPUT_DIR):
    
        clear_output()
    
        print("Unable to find the defined path!")
        exx()
except:
    OUTPUT_DIR = f"{CWD}/downloads/"

if Aria2_rpc:
    if not findProcess("aria2c", "--enable-rpc"):
        try:
            trackers = requests.get("https://trackerslist.com/best_aria2.txt").text
            cmdC = r"aria2c --enable-rpc --rpc-listen-port=6800 -D " \
                   fr"-d {OUTPUT_DIR} " \
                   r"-j 20 " \
                   r"-c " \
                   fr"--bt-tracker={trackers} " \
                   r"--bt-request-peer-speed-limit=0 " \
                   r"--bt-max-peers=0 " \
                   r"--seed-ratio=0.0 " \
                   r"--max-connection-per-server=10 " \
                   r"--min-split-size=10M " \
                   r"--follow-torrent=mem " \
                   r"--disable-ipv6=true " \
                   r" &"
            runSh(cmdC, shell=True)
        except:
            print("aria2 RPC is not enabled! Please enable the RPC first!")

# Configuring port forwarding
clear_output()

if Aria2_rpc:
    Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Aria2_rpc', 6800, 'http']], REGION.lower(), [f"{HOME}/.ngrok2/aria2.yml", 5042])
    data = Server.start('Aria2_rpc', displayB=False)
    Host = urlparse(data['url']).hostname
    port = "80"

clear_output()

if Ariang_WEBUI:
    if Aria2_rpc:
        filePath = 'ariang/index.html'
        with open(filePath, 'r+') as f:
            read_data = f.read()
            f.seek(0)
            f.truncate(0)
            read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', f'rpcHost:"{Host}"', read_data)
            read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
            read_data = re.sub(r'rpcPort:"\d+."', f'rpcPort:"{port}"', read_data)
            f.write(read_data)
    try:
        urllib.request.urlopen(f"http://localhost:{PORT}")
    except:
        runSh(f"python3 -m http.server {PORT} &", shell=True, cd="ariang/")
  
Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Ariang', PORT, 'http']], REGION.lower(), [f"{HOME}/.ngrok2/ariang.yml", 5043])
data2 = Server.start('Ariang', displayB=False)
data2['url'] = urlparse(data2['url'])._replace(scheme='http').geturl()
displayUrl(data2, pNamU='AriaNg : ')

if Aria2_rpc:
    display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>aria2 RPC Configuration<p><thead>        <tr>            <th>Protocol</th>            <th>Host</th>            <th>Port</th>        </tr>    </thead>    <tbody>        <tr>            <td>WebSocket</td><td>"""+Host+"""</td><td>"""+port+"""</td></tr></tbody></table><p>Click <a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="http://bit.ly/2qoOw0a">HERE</a> to see how to use the configuration.</p></button><center>"""))

### <img src="https://upload.wikimedia.org/wikipedia/commons/9/9f/UTorrent_%28logo%29.png" height="45" align="left" alt=""/><font size=6>µTorrent</font>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] µTorrent</font>
# @markdown > µTorrent Default Credential</br>Username: admin</br>Password: admin
USE_FREE_TOKEN = True
TOKEN = ""
REGION = "US"
Tunneling = "argo_tunnel_(cloudflare)" #@param ["argo_tunnel_(cloudflare)", "localhost.run", "ngrok"]

if Tunneling == "argo_tunnel_(cloudflare)":
    PORT_FORWARD = "argotunnel"
elif Tunneling == "localhost.run":
    PORT_FORWARD = "localhost"
elif Tunneling == "ngrok":
    PORT_FORWARD = "ngrok"

# OUTPUT_DIR = ""  # @param {type:"string"}
# ================================================================ #

import os, pathlib, zipfile, re
import urllib.request
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")
r = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
            "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl
)

clear_output()
loadingAn()

# Installing µTorrent
if not os.path.exists("/usr/bin/utserver"):
    os.makedirs("downloads", exist_ok=True)
    r.system_raw("apt install libssl1.0.0 libssl-dev")
    r.system_raw(r"wget http://download-new.utorrent.com/endpoint/utserver/os/linux-x64-ubuntu-13-04/track/beta/ -O utserver.tar.gz")
    r.system_raw(r"tar -zxvf utserver.tar.gz -C /opt/")
    r.system_raw("rm -f utserver.tar.gz")
    r.system_raw("mv /opt/utorrent-server-* /opt/utorrent")
    os.chmod("/opt/utorrent", 0o777)
    r.system_raw("ln -s /opt/utorrent/utserver /usr/bin/utserver")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/configurations/utorrent/utserver.conf", "/opt/utorrent/utserver.conf")

if not findProcess("utserver", "-settingspath"):
    cmd = "utserver -settingspath /opt/utorrent/" \
          " -configfile /opt/utorrent/utserver.conf" \
          " -daemon"
    runSh(cmd, shell=True)

clear_output()

Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['utorrent', 5454, 'http']], REGION.lower, [f"{HOME}/.ngrok2/uTorrent.yml", 4042]).start('utorrent', displayB=False)
displayUrl(Server, pNamU='µTorrent : ', ExUrl=fr"http://{Server['url'][7:]}/gui", btc="g")

# <img src='https://icon-library.com/images/extra-icon/extra-icon-17.jpg' height="30" alt="RcloneLab"/>___Extra___

##<img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/zip.png' height="30" alt="netdata"> __Extracte/Archive Zip/Tar/Rar__

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Extract Files
MODE = "UNZIP"  # @param ["UNZIP", "UNTAR", "UNRAR", "7Z"]
PATH_TO_FILE = "/content/drive/Shareddrives/Love/Ex/miui_POCOF1Global_V12.0.3.0.QEJMIXM_cf3fccffce_10.0"  # @param {type:"string"}
extractPath = "/content/drive/Shareddrives/Love/Ex/Rom"  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# ================================================================ #
import os, urllib.request
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    checkAvailable,
)

def extractFiles():
    global extractPath
    if ARCHIVE_PASSWORD:
      passADD = f'-P {ARCHIVE_PASSWORD}'
    else:
      passADD = ''
    if not extractPath:
      extractPath = "/content/extract"
    os.makedirs(extractPath, exist_ok=True)
    if MODE == "UNZIP":
        runSh('unzip '+passADD+f' "{PATH_TO_FILE}" -d "{extractPath}"', output=True)
    elif MODE == "UNRAR":
        runSh(f'unrar x "{PATH_TO_FILE}" "{extractPath}" '+passADD+' -o+', output=True)
    elif MODE == "UNTAR":
        runSh(f'tar -C "{extractPath}" -xvf "{PATH_TO_FILE}"', output=True)
    else:
        runSh(f'7z x "{PATH_TO_FILE}" -o{extractPath} '+passADD, output=True)


extractFiles()

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Archive Files
MODE = "ZIP" #@param ["ZIP", "TAR", "7Z"]
FILENAME = "UTools"  # @param {type:"string"}
PATH_TO_FILE = "/content/drive/Shareddrives/Telegram/Softwares/O/Tools [4.02].apks"  # @param {type:"string"}
ARCHIVE_PASSWORD = "123" #@param {type:"string"}

# option supports b k m g (bytes, kilobytes, megabytes, gigabytes)
SPLIT = "no" #@param ["1g", "2g", "3g", "4g", "5g", "no"]


compress = 4#@param  {type:"slider", min:0, max:9, step:0}
#@markdown <font size=2%>Use this to path separate between to this character `|`. Ex `path/to /1 | path/to/2` </br>#Split only in zip mode.(Only for noobs)
# ================================================================ #
from pathlib import PurePosixPath

pathList = PATH_TO_FILE.split('|')
if MODE == "ZIP":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE.ZIP"
    if ARCHIVE_PASSWORD:
      passADD = f'--password "{ARCHIVE_PASSWORD}"'
    else:
      passADD = ''
    splitC = f"-s {SPLIT}" if not 'no' in SPLIT else "" 
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && zip {passADD} -{compress} {splitC} -v -r -u "{FILENAME}" "{partName}"'
      !$cmd
elif MODE == "TAR":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE"
    cmd = f'GZIP=-{compress} tar -zcvf "{FILENAME}.tar.gz" {PATH_TO_FILE}'
    !$cmd
else:
    if not FILENAME:
        FILENAME = "/content/NEW_FILE"
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && 7z a -mx={compress} "{FILENAME}.7z" "{partName}"'
      !$cmd


## <img src='https://biplobsd.github.io/RLabClone/img/title_cloud_commander.png' height="30" alt="cloudCommander"/>


In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>⬅️ Install Cloud Commander (file manager)</h3>
#@markdown <br><center><img src='https://biplobsd.github.io/RLabClone/img/title_cloud_commander.png' height="30" alt="netdata"/></center>
# ================================================================ #
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "IN" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost", "argotunnel"]

import os, psutil, time, urllib.request
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    loadingAn,
    PortForward_wrapper,
)

loadingAn()
if os.path.isfile("/tools/node/bin/cloudcmd") == False:
  get_ipython().system_raw("npm cache clean -f && npm install -g n && n stable && npm i cloudcmd -g --force")

try:
  urllib.request.urlopen('http://localhost:7007')
except urllib.error.URLError:
  !nohup cloudcmd --online --no-auth --show-config --show-file-name \
        --editor 'deepword' --packer 'tar' --port 7007 \
        --no-confirm-copy --confirm-move --name 'File Manager' \
        --keys-panel --no-contact --console --sync-console-path \
        --no-terminal --no-vim --columns 'name-size-date' --no-log &
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['netdata', 19999, 'http'], 
                            ['cloudcmd', 7007, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/netdataCloudcmd.yml", 7044]
).start('cloudcmd')

## <img src='https://biplobsd.github.io/RLabClone/img/title_youtube-dl.png' height="30" alt="Youtube-DL"/>

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Execute YouTube-DL
# @markdown 📝 Note: if you want to change an archive file just run this cell again.
Archive = False #@param {type:"boolean"}
# ================================================================ #

import os, uuid, urllib.parse
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files

Links = widgets.Textarea(placeholder='''Video/Playlist Link
(one link per line)''')

VideoQ = widgets.Dropdown(options=["Best Quality (VP9 upto 4K)", "Best Compatibility (H.264 upto 1080p)"])

AudioQ = widgets.Dropdown(options=["Best Quality (Opus)", "Best Compatibility (M4A)"])

Subtitle = widgets.ToggleButton(value=True, description="Subtitle", button_style="info", tooltip="Subtitle")

SavePathYT = widgets.Dropdown(options=["/content", "/content/Downloads"])

AudioOnly = widgets.ToggleButton(value=False, description="Audio Only", button_style="", tooltip="Audio Only")

Resolution = widgets.Select(options=["Highest", "4K", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"], value="Highest")

Extension = widgets.Select(options=["mkv", "webm"], value="mkv")

UsernameYT = widgets.Text(placeholder="Username")

PasswordYT = widgets.Text(placeholder="Password")

SecAuth = widgets.Text(placeholder="2nd Factor Authentication")

VideoPW = widgets.Text(placeholder="Video Password")

GEOBypass = widgets.Dropdown(options=["Disable", "Hide", "AD", "AE", "AF", "AG", "AI", "AL", "AM", "AO", "AQ", "AR", "AS", "AT", "AU", "AW", "AX", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BL", "BM", "BN", "BO", "BQ", "BR", "BS", "BT", "BV", "BW", "BY", "BZ", "CA", "CC", "CD", "CF", "CG", "CH", "CI", "CK", "CL", "CM", "CN", "CO", "CR", "CU", "CV", "CW", "CX", "CY", "CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "EH", "ER", "ES", "ET", "FI", "FJ", "FK", "FM", "FO", "FR", "GA", "GB", "GD", "GE", "GF", "GG", "GH", "GI", "GL", "GM", "GN", "GP", "GQ", "GR", "GS", "GT", "GU", "GW", "GY", "HK", "HM", "HN", "HR", "HT", "HU", "ID", "IE", "IL", "IM", "IN", "IO", "IQ", "IR", "IS", "IT", "JE", "JM", "JO", "JP", "KE", "KG", "KH", "KI", "KM", "KN", "KP", "KR", "KW", "KY", "KZ", "LA", "LB", "LC", "LI", "LK", "LR", "LS", "LT", "LU", "LV", "LY", "MA", "MC", "MD", "ME", "MF", "MG", "MH", "MK", "ML", "MM", "MN", "MO", "MP", "MQ", "MR", "MS", "MT", "MU", "MV", "MW", "MX", "MY", "MZ", "NA", "NC", "NE", "NF", "NG", "NI", "NL", "NO", "NP", "NR", "NU", "NZ", "OM", "PA", "PE", "PF", "PG", "PH", "PK", "PL", "PM", "PN", "PR", "PS", "PT", "PW", "PY", "QA", "RE", "RO", "RS", "RU", "RW", "SA", "SB", "SC", "SD", "SE", "SG", "SH", "SI", "SJ", "SK", "SL", "SM", "SN", "SO", "SR", "SS", "ST", "SV", "SX", "SY", "SZ", "TC", "TD", "TF", "TG", "TH", "TJ", "TK", "TL", "TM", "TN", "TO", "TR", "TT", "TV", "TW", "TZ", "UA", "UG", "UM", "US", "UY", "UZ", "VA", "VC", "VE", "VG", "VI", "VN", "VU", "WF", "WS", "YE", "YT", "ZA", "ZM", "ZW"])

ProxyYT = widgets.Text(placeholder="Proxy URL")

MinSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Min:")

MaxSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Max:")

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def upload_archive():
  if ask_yes_no("Do you already have an archive file? (y/n)", default="", interrupt=""):
    try:
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Please upload an archive from your computer.</h2><br>"))
      UploadConfig = files.upload().keys()
      clear_output(wait=True)
      if len(UploadConfig) == 0:
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
      elif len(UploadConfig) == 1:
        for fn in UploadConfig:
          if os.path.isfile("/content/" + fn):
            get_ipython().system_raw("mv -f " + "\"" + fn + "\" /root/.youtube-dl.txt && chmod 666 /root/.youtube-dl.txt")
            AudioOnly.observe(AudioOnlyChange)
            Subtitle.observe(SubtitleChange)
            AudioQ.observe(AudioQChange)
            ShowYT()
          else:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
      else:
        for fn in UploadConfig:
          get_ipython().system_raw("rm -f " + "\"" + fn + "\"")
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
    except:
      clear_output(wait=True)
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))
  else:
    get_ipython().system_raw("touch '/root/.youtube-dl.txt'")
    AudioOnly.observe(AudioOnlyChange)
    Subtitle.observe(SubtitleChange)
    AudioQ.observe(AudioQChange)
    ShowYT()

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content", "/content/Downloads"]

def AudioOnlyChange(change):
  if change["type"] == "change" and change["new"]:
    VideoQ.disabled = True
    Subtitle.disabled = True
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = True
    Extension.options = ["best", "aac", "flac", "mp3", "m4a", "opus", "vorbis", "wav"]
    Extension.value = "best"
    AudioOnly.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    VideoQ.disabled = False
    Subtitle.disabled = False
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = False
    if AudioQ.value == "Best Quality (Opus)":
      Extension.options = ["mkv", "webm"]
    else:
      Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"
    AudioOnly.button_style = ""

def SubtitleChange(change):
  if change["type"] == "change" and change["new"]:
    Subtitle.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    Subtitle.button_style = ""

def AudioQChange(change):
  if change["type"] == "change" and change["new"] == "Best Quality (Opus)":
    Extension.options = ["mkv", "webm"]
    Extension.value = "mkv"
  elif change["type"] == "change" and change["new"] == "Best Compatibility (M4A)":
    Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"

def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  display(widgets.HBox([widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      widgets.HTML("<b style=\"color:#888888;\">For website that require an account:</b>"), UsernameYT, PasswordYT, SecAuth, VideoPW,
                                      widgets.HTML("<b><a href=\"https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Officially_assigned_code_elements\" target=\"_blank\">GEO Bypass Country:</a></b>"), GEOBypass,
                                      widgets.HTML("<b style=\"color:#888888;\">Proxy:</b>"), ProxyYT,
                                      widgets.HTML("<b style=\"color:#888888;\">Sleep Interval (second):</b>"), MinSleep, MaxSleep]),
                        widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Video Quality:</b>"), VideoQ, widgets.HTML("<b style=\"color:#888888;\">Resolution:</b>"), Resolution,
                                      widgets.HTML("<b style=\"color:#888888;\">Audio Quality:</b>"), AudioQ, widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Options:</b>"), widgets.HBox([Subtitle, AudioOnly]),
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), ExtraArg])]), HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"),
          SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    # Account Check
    if UsernameYT.value.strip() and PasswordYT.value.strip():
      accountC = "--username \"" + UsernameYT.value + "\" --password \"" + PasswordYT.value + "\""
    else:
      accountC = ""
    if SecAuth.value.strip():
      secauthC = "-2 " + SecAuth.value
    else:
      secauthC = ""
    if VideoPW.value.strip():
      videopwC = "--video-password " + VideoPW.value
    else:
      videopwC = ""
    # Proxy
    if ProxyYT.value.strip():
      proxyytC = "--proxy " + ProxyYT.value
    else:
      proxyytC = ""
    # GEO Bypass
    if GEOBypass.value == "Disable":
      geobypass = ""
    elif GEOBypass.value == "Hide":
      geobypass = "--geo-bypass"
    else:
      geobypass = "--geo-bypass-country " + GEOBypass.value
    # Video Quality
    if VideoQ.value == "Best Quality (VP9 upto 4K)":
      videoqC = "webm"
    else:
      videoqC = "mp4"
    # Audio Quality
    if AudioQ.value == "Best Quality (Opus)":
      audioqC = "webm"
    else:
      audioqC = "m4a"
    # Audio Only Check
    if AudioOnly.value:
      subtitleC = ""
      thumbnailC = ""
      extC = "-x --audio-quality 0 --audio-format " + Extension.value
      codecC = "bestaudio[ext=" + audioqC + "]/bestaudio/best"
    else:
      if Subtitle.value:
        subtitleC = "--all-subs --convert-subs srt --embed-subs"
      else:
        subtitleC = ""
      if Extension.value == "mp4":
        thumbnailC = "--embed-thumbnail"
      else:
        thumbnailC = ""
      extC = "--merge-output-format " + Extension.value
      if Resolution.value == "Highest":
        codecC = "bestvideo[ext=" + videoqC + "]+bestaudio[ext=" + audioqC + "]/bestvideo+bestaudio/best"
      else:
        codecC = "bestvideo[ext=" + videoqC + ",height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio[ext=" + audioqC + "]/bestvideo[height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio/bestvideo+bestaudio/best"
    # Archive
    if os.path.isfile("/root/.youtube-dl.txt"):
      archiveC = "--download-archive \"/root/.youtube-dl.txt\""
    else:
      archiveC = ""
    # Sleep Interval
    if MinSleep.value > 0 and MaxSleep.value > 0:
      minsleepC = "--min-sleep-interval " + MinSleep.value
      maxsleepC = "--max-sleep-interval " + MaxSleep.value
    else:
      minsleepC = ""
      maxsleepC = ""
    # Extra Arguments
    extraargC = ExtraArg.value
    for Link in Links.value.splitlines():
      clear_output(wait=True)
      Count += 1
      display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Processing link " + str(Count) + " out of " + Total + "</h3>"))
      if "youtube.com" in Link or "youtu.be" in Link:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading...</h3><br>"), YouTubeVideo(Link, width=640, height=360), HTML("<br>"))
      else:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading <a href=\"" + Link + "\">" + Link + "</a></h3><br>"))
      if ("youtube.com" in Link or "youtu.be" in Link) and "list=" in Link:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s" "$Link"
      else:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(title)s.%(ext)s" "$Link"
      if not os.path.exists(SavePathYT.value):
        get_ipython().system_raw("mkdir -p -m 666 " + SavePathYT.value)
      get_ipython().system_raw("mv /root/.YouTube-DL/* '" + SavePathYT.value + "/'")
    # Archive Download
    if os.path.isfile("/root/.youtube-dl.txt"):
      files.download("/root/.youtube-dl.txt")
    ShowYT()

if not os.path.isfile("/usr/local/bin/youtube-dl"):
  get_ipython().system_raw("rm -rf /content/sample_data/ && mkdir -p -m 666 /root/.YouTube-DL/ && apt-get install atomicparsley && curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl && chmod a+rx /usr/local/bin/youtube-dl")
if Archive:
  upload_archive()
else:
  AudioOnly.observe(AudioOnlyChange)
  Subtitle.observe(SubtitleChange)
  AudioQ.observe(AudioQChange)
  ShowYT()

## <img src="https://lh3.googleusercontent.com/eVQcNGH3ydrVOr6T1N4vdVa3QXtCxb3Mgp6Lt3p0wrN63GM4FFgGOgu-pBIZA4r8R2c" height="45" align="left" alt=""/><font size=6>Unzip</font>

In [ ]:
%cd /content/drive/Shareddrives/

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!unzip "/content/drive/Shareddrives/" -d "/content/drive/Shareddrives/"

# <img src='https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/Icons8_flat_upload.svg/800px-Icons8_flat_upload.svg.png' height="30" alt="RcloneLab"/>___Remote Uploading___

In [ ]:
#@markdown <h3>⬅️ Click Here to START SERVER</h3>

import os

%cd /content/drive/My Drive
os.makedirs("DirectDownload", exist_ok=True)
%cd DirectDownload

#####################################

%pip install wget

#####################################
import wget
url = input("Enter Remote Url")
initiateDownload = wget.download(url)
initiateDownload

##

%ls /content/drive/My\ Drive/DirectDownload



## __Remote Uploader__

__Supported filehosts:__
*   MediaFire
*   Zippyshare
*   anonfiles
*   BayFiles



In [ ]:
#@title #__👈 Run this cell to install requirements__##
!echo -n "Installing bayfiles-cli..." && npm i -g bayfiles-cli &> /dev/null && echo " Done!"
!echo -n "Installing anonfile-cli..." && mkdir -p sample_data/anonfile-upload &> /dev/null && git clone https://github.com/awersli99/anonfile-upload.git sample_data/anonfile-upload &> /dev/null && cd sample_data/anonfile-upload &> /dev/null && pip install -r requirements.txt &> /dev/null && echo " Done!"
!echo -n "Installing zippyshare-ul..." && mkdir -p sample_data/zs-ul &> /dev/null && git clone https://github.com/Sorrow446/ZS-UL.git sample_data/zs-ul &> /dev/null && cd sample_data/zs-ul &> /dev/null && pip install -r requirements.txt &> /dev/null && echo " Done!"
!echo -n "Installing zippyshare-dl..." && mkdir -p sample_data/zs-dl &> /dev/null && git clone https://github.com/Sorrow446/ZS-DL.git sample_data/zs-dl &> /dev/null && cd sample_data/zs-dl &> /dev/null && pip install -r requirements.txt &> /dev/null && echo " Done!"
!echo -n "Installing mediafire-ul..." && mkdir -p sample_data/mediafire-ul &> /dev/null && git clone https://github.com/JayBrown/mfcmd.py---MediaFire-uploader.git sample_data/mediafire-ul &> /dev/null && pip install pymediafire &> /dev/null && pip install mediafire &> /dev/null && echo " Done!"

## __Download/Upload__

In [ ]:
#@title #__⬇️ Download from filehosts to cloud storage services__#
#@markdown __Supported filehosts:__
#@markdown *   MediaFire (https://download2280.mediafire.com/xxx...)
#@markdown *   Zippyshare (https://www5.zippyshare.com/v/xxx...)
#@markdown *   anonfiles (https://anonfiles.com/xxx...)
#@markdown *   BayFiles (https://bayfiles.com/xxx...)
URL = "https://download1584.mediafire.com/7r2ncvtqnvsg/m2a2dtdp27gs6ca/Win7_Pro_SP1_English_x64.iso" #@param {type:"string"}
Download_Destination = "/content/drive/Shareddrives/Random/Softwares/" #@param {type:"string"}
url, downdir = URL, Download_Destination

def get_file_info(url):
  splited = url.split("/")
  filename=splited[-1]
  splitedfile = filename.split("_")
  splitedfile2="_".join(splitedfile[0:-1])
  modfilename=splitedfile2+"."+splitedfile[-1]
  return filename, modfilename

if "zippyshare" in url:
  !cd sample_data/zs-dl && python3 zs-dl.py -u $url -o $downdir > /content/sample_data/output.txt
  !tail -n 2 sample_data/output.txt
elif "mediafire" in url:
  !wget $url -P $downdir
elif "anonfiles" or "bayfiles" in url:
  filename, modfilename = get_file_info(url)
  !bayfilesdownload $url -o $downdir
  !echo -n "Please correct the file extention on your own: file_txt -> file.txt"
else:
  print("Error: Unsupported filehost")

In [ ]:
#@title #__⬆️ Upload to filehosts from cloud storage services__#
#@markdown __Successfully uploaded max file size:__
#@markdown *   anonfiles (1950MB)
#@markdown *   Zippyshare (500MB)
#@markdown *   BayFiles (99MB)
FileHost = "anonfiles" #@param ["Zippyshare", "anonfiles", "BayFiles"]
File_Path = "" #@param {type:"string"}
filehost, infile = FileHost, File_Path

def get_file_info(filepath):
  splited = filepath.split("/")
  filename=splited[-1]
  pathname="/".join(splited[0:-1])
  return filename, pathname

if filehost == "Zippyshare":
  filename, pathname = get_file_info(infile)
  print("Uploading", filename+"...")
  !cd sample_data/zs-ul && python3 zs-ul.py -f "$infile" #> /content/sample_data/output.txt
  #!tail -n 1 sample_data/output.txt
if filehost == "anonfiles":
  filename, pathname = get_file_info(infile)
  print("Uploading", filename+"...")
  !cd sample_data/anonfile-upload && python3 upload.py "$infile" #> /content/sample_data/output.txt
  #!tail -n 2 sample_data/output.txt
if filehost == "BayFiles":
  filename, pathname = get_file_info(infile)
  print(filename, pathname)
  print("Uploading", filename+"...")
  !cd "$pathname" && bayfilesupload "$filename" #> /content/sample_data/output.txt
  #!tail -n 1 sample_data/output.txt

In [ ]:
#@title #__⬆️ MediaFire Uploader__#
#@markdown The MediaFire API is pretty buggy, and you might receive an error, because the server hasn't finished calculating the file's checksum yet; in that case, wait a couple of seconds (depending on the filesize) and repeat the command: it will not upload, but instead detect the already uploaded file and return the download URL.
Email_Address = "" #@param {type:"string"}
Password = "" #@param {type:"string"}
File_path = "" #@param {type:"string"}
mfemail, mfpassword = Email_Address, Password

def get_file_info(filepath):
  splited = filepath.split("/")
  filename=splited[-1]
  pathname="/".join(splited[0:-1])
  return filename, pathname

filename, pathname = get_file_info(File_path)
print("Uploading", filename+"...")
!cd /content/sample_data/mediafire-ul && python3 mfcmd.py -e $mfemail -p $mfpassword -f $File_path 2>/dev/null
print("Done!")